In [143]:
import numpy as np
import pandas as pd
import requests
from selenium import webdriver
import re
import math
from bs4 import BeautifulSoup
from multiprocessing.dummy import Pool as ThreadPool
from time import time

In [137]:
def get_pages_count(item):
    items_in_table = 30
    count = item.text
    count = re.sub(re.compile(r'\s+'), '', count)
    return math.ceil(int(count) / items_in_table)
    

In [138]:
def get_serials_on_page(page_id):
    url_template = "https://myshows.me/search/all/?page={}"
    serial_list_url = url_template.format(page_id)
    serial_list_html = requests.get(serial_list_url).text.strip()
    soup = BeautifulSoup(serial_list_html, 'html.parser')
    print(page_id)
    serials = {}
    table = soup.findAll('table', class_='catalogTable')
    items = table[1].findAll('tr')[1:]
    for item in items:
        serial_info = item.find('td').find('a')
        serial_name = serial_info.text.strip()
        serial_url = serial_info.attrs['href']
        serials[serial_name] = re.findall(r'\d+', serial_url)[0]
    return serials

In [139]:
def get_url_list():
    url_template = "https://myshows.me/search/all/?page={}"
    serial_list_url = url_template.format(0)
    serial_list_html = requests.get(serial_list_url).text.strip()
    soup = BeautifulSoup(serial_list_html, 'html.parser')
    pages_count = get_pages_count(soup.find('span', class_='cFirm'))
    
    url_list = []
    t = ThreadPool(8)
    result = t.map(get_serials_on_page, range(pages_count))
    t.close()
    t.join()
    
    return result

In [140]:
%%time

serial_list = get_url_list()

0
210168

294
126
252
42
84
1
169
43
85
295
211
253
2127

170
44
296
212
128
2543

86
171
45
213
4
255
87
129
297
172
46
5
88
256
130
173
47
214
298
6
89
17448215


257
299
131
7
90216

132
300
175
49
258
8
217
91
176
301
50
133
259
9
218
177
92
302
51
134
260
10
219
178
93
11
52
303
261
135
179
220
94
12
180304

53
136
221
262
13
95
54
137
181
22214

263
305
96
55
138
182
264
306
56
223
97
15
139
183
265
98
224
307
57
184
16140

266
99225308


58
267
185
17
141
100
59226

309
18
142
268
186
101
60
310227

187
19
269
143
102
61
311
20
228
103
270
188
144
62
21
312
145
189
63
104
229
271
313
22
146
64
190
105
230
272
314
23
147
191
65
106
231
24
315
273
148
192
66
107
25149
316

232
274
67
193
108
26
275
68
233150

194
317
109
27
234
151
69
276
195
318
110
28
70
277
235
152
319
196
111
29
71
153
278
30
112
197
320
236
72
31
113
198
321
279
73
154
237
32
322
199
33
114
74
238
280155

323
200
34
239
115
75281

156
324
201
35
76
282
157
116
240
325
36
202
77
283
158
117
326
37
241
203
78
2

In [157]:
serials = serial_list[0]
for serial_dict in serial_list[1:]:
    serials.update(serial_dict)

data = pd.DataFrame(list(serials.items()), columns=['Name', 'ID'])
with open('serials.csv', mode='w', encoding='utf-8') as f_csv:
    data.to_csv(f_csv, index=False)